In [ ]:
#=
tracer release at single location in periodic basin with random wind stress
simulation runs for 365 days and tracer is averaged and saved every hour

author: Luke Gloege
date: 2024-12-20
=#

In [1]:
using Pkg
Pkg.activate(".")

using Dates
using Random
using Printf

using Oceananigans
using Oceananigans.Units: minute, minutes, hour, hours, day, days, meter, meters, kilometer, kilometers


  Activating project at `/gpfs/gibbs/project/eisaman/ka659/pinns-oceananigans-project/oceananigans/simulations`
[ Info: Precompiling Oceananigans [9e8cae18-63c1-5223-a75c-80ca9d6e9a09]


In [2]:
# --------------------------------------------------
# computing architecture
# --------------------------------------------------
architecture = CPU()

CPU()

In [3]:
# --------------------------------------------------
# grid
# --------------------------------------------------
const Nx = 100  # number of x points 
const Ny = 100  # number of y points 
const Nz = 1    # number of z points 

const Lx = 100kilometers # (m) domain x extent
const Ly = 100kilometers # (m) domain y extent
const Lz = 10meters      # (m) domain depth

grid = RectilinearGrid(
    architecture, 
    topology = (Periodic, Periodic, Bounded),
    size = (Nx, Nx, Nz),
    halo = (3,3,2),
    x = (0, Lx),
    y = (0, Ly),
    z = (-Lz, 0)
)

100×100×1 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 3×3×2 halo
├── Periodic x ∈ [0.0, 100000.0) regularly spaced with Δx=1000.0
├── Periodic y ∈ [0.0, 100000.0) regularly spaced with Δy=1000.0
└── Bounded  z ∈ [-10.0, 0.0]    regularly spaced with Δz=10.0

In [4]:
# --------------------------------------------------
# coriolis
# --------------------------------------------------
coriolis = FPlane(f=1e-4)

# --------------------------------------------------
# closure
# --------------------------------------------------
closure = AnisotropicMinimumDissipation()

# --------------------------------------------------
# seawater buoyancy
# --------------------------------------------------
buoyancy = SeawaterBuoyancy(
    equation_of_state = LinearEquationOfState(
        thermal_expansion = 2e-4,
        haline_contraction = 8e-4)
)


SeawaterBuoyancy{Float64}:
├── gravitational_acceleration: 9.80665
└── equation_of_state: LinearEquationOfState(thermal_expansion=0.0002, haline_contraction=0.0008)

In [5]:
# --------------------------------------------------
# surface stress
# --------------------------------------------------
u₁₀ = 10    # m s⁻¹, average wind velocity 10 meters above the ocean
ρₒ = 1026.0 # kg m⁻³, average density at the surface of the world ocean
cᴰ = 2.5e-3 # dimensionless drag coefficient
ρₐ = 1.225  # kg m⁻³, average density of air at sea-level
  
@inline wind_stress_x(x, y, t, p) = (- ρₐ / ρₒ * cᴰ * u₁₀ * abs(u₁₀)) * rand()
@inline wind_stress_y(x, y, t, p) = (- ρₐ / ρₒ * cᴰ * u₁₀ * abs(u₁₀)) * rand()


wind_stress_y (generic function with 1 method)

In [6]:
# --------------------------------------------------
# boundary conditions
# --------------------------------------------------
u_top_bcs = FluxBoundaryCondition(wind_stress_x, parameters=(k=4π, ω=3.0, τ=1e-4))
v_top_bcs = FluxBoundaryCondition(wind_stress_y, parameters=(k=4π, ω=3.0, τ=1e-4))

u_bcs = FieldBoundaryConditions(top = u_top_bcs) 
v_bcs = FieldBoundaryConditions(top = v_top_bcs)

T_bcs = FieldBoundaryConditions()
S_bcs = FieldBoundaryConditions()
c_bcs = FieldBoundaryConditions()

Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── top: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)

In [7]:
# --------------------------------------------------
# instantiate model
# --------------------------------------------------
model = NonhydrostaticModel(; grid, buoyancy,
                            timestepper = :QuasiAdamsBashforth2,
                            advection = WENO(),
                            tracers = (:T, :S, :c),
                            coriolis = coriolis,
                            closure = closure,
                            boundary_conditions = (u=u_bcs, v=v_bcs, T=T_bcs, S=S_bcs, c=c_bcs))



[ Info: Using the advection scheme UpwindBiased(order=1) in the z-direction because size(grid, 3) = 1


NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 100×100×1 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 3×3×2 halo
├── timestepper: QuasiAdamsBashforth2TimeStepper
├── advection scheme: FluxFormAdvection(x=WENO(order=5), y=WENO(order=5), z=UpwindBiased(order=1))
├── tracers: (T, S, c)
├── closure: AnisotropicMinimumDissipation{Oceananigans.TurbulenceClosures.ExplicitTimeDiscretization, NamedTuple{(:T, :S, :c), Tuple{Float64, Float64, Float64}}, Float64, Nothing}
├── buoyancy: SeawaterBuoyancy with g=9.80665 and LinearEquationOfState(thermal_expansion=0.0002, haline_contraction=0.0008) with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [8]:
# --------------------------------------------------
# initial conditions
# --------------------------------------------------
const u_initial = 0.0
const v_initial = 0.0
const T_initial = 20.0
const S_initial = 35.0

# initial concentration value
# center indices (x, y) 
center_x_index = Int(grid.Nx / 2)  # center index in the x-direction
center_y_index = Int(grid.Ny / 2)  # center index in the y-direction
surface_z_index = grid.Nz          # index of surface z grid cell 

# initialize tracer values to zero
c_initial = model.tracers.c 

# grid cell volume
# note: this assumes grid cells all same size
dx = Lx / Nx       # (m) grid cell x length 
dy = Ly / Ny       # (m) grid cell y length
dz = Lz / Nz       # (m) grid cell z length
dv = dx * dy * dz  # (m3) grid cell volume

# set initial concentration to 1 mol / m3 at center grid cell
c_initial[center_x_index, center_y_index, surface_z_index] = 1

set!(model, u=u_initial, v=v_initial, T=T_initial, S=S_initial, c=c_initial)


In [9]:
# --------------------------------------------------
# simulation and outputs
# --------------------------------------------------
simulation = Simulation(model, Δt=10minutes, stop_time=100days)

wizard = TimeStepWizard(cfl=0.2)

simulation.callbacks[:wizard] = Callback(wizard, IterationInterval(10))


progress_message(sim) = @printf("Iteration: %04d, time: %s, Δt: %s, max(|u|) = %.1e ms⁻¹, wall time: %s\n",
                                iteration(sim), prettytime(sim), prettytime(sim.Δt),
                                maximum(abs, sim.model.velocities.u), prettytime(sim.run_wall_time))

add_callback!(simulation, progress_message, IterationInterval(20))



In [10]:
# --------------------------------------------------
# output writer
# --------------------------------------------------
filename = "output-tracer-release_$(string(today()))"

simulation.output_writers[:surface_slice_writer] = NetCDFOutputWriter(
    model, 
    (; model.velocities.u, model.velocities.v, model.tracers.S, model.tracers.T, model.tracers.c),
    filename = filename * ".nc",
    schedule = AveragedTimeInterval(1hour, window=1hour),
    indices=(:, :, grid.Nz),
    overwrite_existing = true
)


NetCDFOutputWriter scheduled on TimeInterval(1 hour):
├── filepath: output-tracer-release_2025-06-06.nc
├── dimensions: zC(1), zF(1), xC(100), yF(100), xF(100), yC(100), time(0)
├── 5 outputs: (v, S, c, T, u) averaged on AveragedTimeInterval(window=1 hour, stride=1, interval=1 hour)
└── array type: Array{Float64}
├── file_splitting: NoFileSplitting
└── file size: 21.4 KiB

In [11]:
# --------------------------------------------------
# run simulation
# --------------------------------------------------
run!(simulation)

[ Info: Initializing simulation...


Iteration: 0000, time: 0 seconds, Δt: 11 minutes, max(|u|) = 0.0e+00 ms⁻¹, wall time: 0 seconds


[ Info:     ... simulation initialization complete (22.436 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (5.990 seconds).


Iteration: 0020, time: 3.605 hours, Δt: 9.465 minutes, max(|u|) = 3.1e-01 ms⁻¹, wall time: 29.141 seconds
Iteration: 0040, time: 6.123 hours, Δt: 6.100 minutes, max(|u|) = 4.1e-01 ms⁻¹, wall time: 29.667 seconds
Iteration: 0060, time: 8 hours, Δt: 5.211 minutes, max(|u|) = 3.7e-01 ms⁻¹, wall time: 30.179 seconds
Iteration: 0080, time: 9.689 hours, Δt: 5.320 minutes, max(|u|) = 2.7e-01 ms⁻¹, wall time: 30.699 seconds
Iteration: 0100, time: 11.472 hours, Δt: 6.225 minutes, max(|u|) = 1.4e-01 ms⁻¹, wall time: 31.211 seconds
Iteration: 0120, time: 13.457 hours, Δt: 7.532 minutes, max(|u|) = 6.0e-02 ms⁻¹, wall time: 31.739 seconds
Iteration: 0140, time: 15.967 hours, Δt: 9.114 minutes, max(|u|) = 1.0e-01 ms⁻¹, wall time: 32.250 seconds
Iteration: 0160, time: 18.835 hours, Δt: 11.028 minutes, max(|u|) = 1.5e-01 ms⁻¹, wall time: 32.772 seconds
Iteration: 0180, time: 22 hours, Δt: 8.303 minutes, max(|u|) = 3.7e-01 ms⁻¹, wall time: 33.317 seconds
Iteration: 0200, time: 1.009 days, Δt: 5.574 min

[ Info: Simulation is stopping after running for 13.273 minutes.
[ Info: Simulation time 100 days equals or exceeds stop time 100 days.
